In [35]:
import os
import numpy as np
import shutil

In [36]:
os.chdir("E://Insofe//Python Lab//10_class_ic_input/")

In [37]:
train_dir = "E://Insofe//Python Lab//10_categories/"

In [38]:
# Create folders for  training data and validation randomly splitted 
new_train = os.mkdir("new_train")
new_valid = os.mkdir("new_valid")

In [39]:
new_train_dir = "E:/Insofe/Python Lab/10_class_ic_input/new_train/"
new_valid_dir = "E:/Insofe/Python Lab/10_class_ic_input/new_valid/"

In [40]:
new_train_dir

'E:/Insofe/Python Lab/10_class_ic_input/new_train/'

In [41]:
new_valid_dir

'E:/Insofe/Python Lab/10_class_ic_input/new_valid/'

In [42]:
classes = os.listdir(train_dir) # Get all the classes from the original train directory

In [43]:
classes

['airplanes',
 'BACKGROUND_Google',
 'bonsai',
 'car_side',
 'Faces',
 'Faces_easy',
 'grand_piano',
 'Leopards',
 'Motorbikes',
 'watch']

In [44]:
# Create target folders in the new train data directory
for c in classes:
    os.mkdir(new_train_dir+c)

In [45]:
# Create validation folders in the new validation data directory
for c in classes:
    os.mkdir(new_valid_dir+c)

In [46]:
data_path= train_dir+classes[0]+'/'

In [47]:
for species in classes:
    data_root = train_dir+species+'/'
    data_new_train = new_train_dir + species + '/'
    data_new_valid = new_valid_dir + species + '/'
    data_root_list = os.listdir(data_root)
    test_size = 0.8
    split_index = int(0.8*len(data_root_list))
    np.random.shuffle(data_root_list)
    train_list = data_root_list[:split_index]
    valid_list = data_root_list[split_index:]
    for image in train_list :
        source = data_root + image
        target = data_new_train + image
        shutil.copy(source,target)
    for image in valid_list:
        source = data_root + image
        target = data_new_valid + image
        shutil.copy(source,target)


In [48]:
train_no = []
valid_no = []
for c in classes:
    train_no.append((len(os.listdir(new_train_dir+c+'/'))))
    valid_no.append(len(os.listdir(new_valid_dir+c+'/')))

In [49]:
sum(train_no)

2977

In [50]:
sum(valid_no)

747

In [51]:
train_airplanes_dir = "E://Insofe//Python Lab//10_class_ic_input//new_train//airplanes/"
validation_airplanes_dir = "E://Insofe//Python Lab//10_class_ic_input//new_valid/airplanes/"
print('total training airplane images:', len(os.listdir(train_airplanes_dir)))
print('total validation airplane images:', len(os.listdir(validation_airplanes_dir)))

total training airplane images: 640
total validation airplane images: 160


In [52]:
train_BACKGROUND_Google_dir = "E://Insofe//Python Lab//10_class_ic_input//new_train/BACKGROUND_Google/"
validation_BACKGROUND_Google_dir = "E://Insofe//Python Lab//10_class_ic_input//new_valid/BACKGROUND_Google/"
print('total train BACKGROUND_Google  images:', len(os.listdir(train_BACKGROUND_Google_dir)))
print('total validation validation airplane images:', len(os.listdir(validation_BACKGROUND_Google_dir)))

total train BACKGROUND_Google  images: 373
total validation validation airplane images: 94


In [53]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10,activation='softmax'))


In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [55]:
from keras import optimizers
model.compile(optimizer='RMSprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Data Preprocessing

As you know by now, data should be formatted into appropriately preprocessed floatingpoint tensors before being fed into the network. Currently, the data sits on a drive as
JPEG files, so the steps for getting it into the network are roughly as follows:
1 Read the picture files.
2 Decode the JPEG content to RGB grids of pixels.
3 Convert these into floating-point tensors.
4 Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know,
neural networks prefer to deal with small input values).
It may seem a bit daunting, but fortunately Keras has utilities to take care of these
steps automatically. Keras has a module with image-processing helper tools, located at
keras.preprocessing.image. In particular, it contains the class ImageDataGenerator,
which lets you quickly set up Python generators that can automatically turn image files
on disk into batches of preprocessed tensors. This is what you’ll use here.

### Using ImageDataGenerator to read images from directories

In [56]:
train_dir = "E:\\Insofe\\Python Lab\\10_class_ic_input\\new_train\\"

In [57]:
validation_dir = "E:\\Insofe\\Python Lab\\10_class_ic_input\\new_valid\\"

In [58]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              target_size=(150, 150),
                                                              batch_size=20,
                                                              class_mode='categorical')

Found 2977 images belonging to 10 classes.
Found 747 images belonging to 10 classes.


In [59]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20, 10)


## Fitting the model using a batch generator

In [26]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=25,
                              validation_data=validation_generator,
                              validation_steps=100)

Epoch 1/25
100/100 [==============================] - 138s 1s/step - loss: 1.6968 - acc: 0.4625 - val_loss: 0.8614 - val_acc: 0.7138
Epoch 2/25
100/100 [==============================] - 133s 1s/step - loss: 0.6285 - acc: 0.8003 - val_loss: 0.5074 - val_acc: 0.8338
Epoch 3/25
100/100 [==============================] - 133s 1s/step - loss: 0.4019 - acc: 0.8754 - val_loss: 0.3295 - val_acc: 0.8972
Epoch 4/25
100/100 [==============================] - 134s 1s/step - loss: 0.2979 - acc: 0.9065 - val_loss: 0.3468 - val_acc: 0.8883
Epoch 5/25
100/100 [==============================] - 104s 1s/step - loss: 0.1888 - acc: 0.9444 - val_loss: 0.4532 - val_acc: 0.9057
Epoch 6/25
100/100 [==============================] - 93s 932ms/step - loss: 0.1793 - acc: 0.9490 - val_loss: 0.2410 - val_acc: 0.9291
Epoch 7/25
100/100 [==============================] - 93s 927ms/step - loss: 0.1119 - acc: 0.9630 - val_loss: 0.3552 - val_acc: 0.9250
Epoch 8/25
100/100 [==============================] - 93s 930ms/s

### Saving the model

In [31]:
model.save('Image_Classification_high_epochs.h5')

### Displaying curves of loss and accuracy during training

In [27]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show();

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [28]:
# Prepare Testing Data
import pandas as pd
test_filenames = os.listdir("E:\\Insofe\\Python Lab\\10_class_ic_output\\test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [29]:
test_df

,filename
0,test_image-1551435475921.jpg


In [30]:
test_dir = "E:\\Insofe\\Python Lab\\10_class_ic_output\\"

In [31]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                              target_size=(150, 150),
                                                              batch_size=20,
                                                              class_mode='categorical')

Found 1 images belonging to 1 classes.


In [32]:
test_dir

'E:\\Insofe\\Python Lab\\10_class_ic_output\\'

In [33]:
# Predict
batch_size=20
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

In [34]:
predict

array([[9.99894977e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.05013576e-04, 0.00000000e+00]], dtype=float32)

In [35]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [36]:
predict

array([[0.999895, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000,
        0.000000, 0.000000, 0.000105, 0.000000]], dtype=float32)

In [37]:
y_pred = pd.DataFrame(predict, columns=[classes])

In [38]:
y_pred

,airplanes,BACKGROUND_Google,bonsai,car_side,Faces,Faces_easy,grand_piano,Leopards,Motorbikes,watch
0,0.999895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000105,0.0


In [39]:
y_pred.head()

,airplanes,BACKGROUND_Google,bonsai,car_side,Faces,Faces_easy,grand_piano,Leopards,Motorbikes,watch
0,0.999895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000105,0.0


In [40]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [41]:
y_pred.head()

,airplanes,BACKGROUND_Google,bonsai,car_side,Faces,Faces_easy,grand_piano,Leopards,Motorbikes,watch
0,0.999895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000105,0.0


In [42]:
y_pred['Max'] = y_pred.idxmax(axis=1)

In [43]:
 y_pred.head(1)

,airplanes,BACKGROUND_Google,bonsai,car_side,Faces,Faces_easy,grand_piano,Leopards,Motorbikes,watch,Max
0,0.999895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000105,0.0,"(airplanes,)"


In [27]:
#Transfer Learning
# Inception was trained on 299x299x3 size images. Therefore we'll use the same dimentions for out task.
# Nevertheless, in transfer learning, if we are importing only the convolution layers, we can resize the images to other resolutions as well
# Inception is a fairly big network. Hence we used batch size as 64. 
# If larger memory GPUs are available, bigger batch_size could be used.

train_datagen = ImageDataGenerator(rescale=1/255.,
                                    rotation_range=20,
                                    height_shift_range=0.2,
                                    zoom_range=0.2)
train_generator = train_datagen.flow_from_directory(directory=train_dir, 
                                                    batch_size=64, 
                                                    class_mode='categorical', 
                                                    shuffle=True, 
                                                    target_size=(299,299))

Found 2977 images belonging to 10 classes.


In [28]:
val_datagen = ImageDataGenerator(rescale=1/255.)
val_generator = train_datagen.flow_from_directory(directory=validation_dir, 
                                                    batch_size=64, 
                                                    class_mode='categorical', 
                                                    shuffle=False, 
                                                    target_size=(299,299))

Found 747 images belonging to 10 classes.


In [29]:
from keras.models import Sequential, Model, load_model
from keras.layers import *
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.utils import to_categorical
import cv2
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


In [30]:
# Get Inception architecture from keras.applications
from keras.applications.inception_v3 import InceptionV3
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.core import Flatten, Dense, Dropout, Lambda
def inception_tl(nb_classes, freez_wts):
    
    trained_model = InceptionV3(include_top=False,weights='imagenet')
    x = trained_model.output
    x = GlobalAveragePooling2D()(x)
    pred_inception= Dense(nb_classes,activation='softmax')(x)
    model = Model(inputs=trained_model.input,outputs=pred_inception)
    
    for layer in trained_model.layers:
        layer.trainable=(1-freez_wts)
    
    return(model)

In [ ]:
batch_size = 20
model = inception_tl(nb_classes=10, freez_wts=False)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
hist1 = model.fit_generator(train_generator, 
                           validation_data=val_generator, 
                           steps_per_epoch=sum(train_no)//batch_size,
                           validation_steps=sum(valid_no)//batch_size,
                           epochs=20).history

Epoch 1/20
148/148 [==============================] - 13884s 94s/step - loss: 0.2233 - acc: 0.9379 - val_loss: 3.2038 - val_acc: 0.4195
Epoch 2/20
  7/148 [>.............................] - ETA: 2:55:47 - loss: 0.1438 - acc: 0.9464

In [ ]:
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
train_loss = plt.plot(hist1['loss'], label='train loss')
val_loss = plt.plot(hist1['val_loss'], label='val loss')
plt.legend()
plt.title('Loss')

plt.subplot(1,2,2)
train_loss = plt.plot(hist1['acc'], label='train acc')
val_loss = plt.plot(hist1['val_acc'], label='val acc')
plt.legend()
plt.title('Accuracy')

In [ ]:
## Learning Rate Scheduler
model = inception_tl(nb_classes=10, freez_wts=False)
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=adam)

In [ ]:
nb_epochs = 10
init_lr = 0.001
min_lr = 0.000001
f = 10**(np.log10(min_lr/init_lr)/float(nb_epochs))

def poly_decay(epoch):
    ''' This function takes the current epoch as input and return the updated learning rate.
        The learning rate is multiplied by a factor 'f' after each epoch.
        In the first epoch, learning rate is set to 'init_lr'.
        By the end of 'nb_epochs' the learning rate is reduced to 'min_lr' '''
    return(init_lr*(f**epoch))

# ModelCheckpoint monitors the 'val_loss' and saves the model graph and weights at the epoch with least 'val_loss'
# 'save_weights_only'=True, saves only the weights
# 'save_weights_only'=False, saves the weights and the graph
chkp = ModelCheckpoint(filepath='inception_dd.h5', monitor='val_loss', save_best_only=True, save_weights_only=False, verbose=0)
lr_schedule = LearningRateScheduler(poly_decay)

In [ ]:
lr_list = [poly_decay(i) for i in range(nb_epochs)]
plt.scatter(range(nb_epochs), np.log10(lr_list))
plt.plot(np.log10(lr_list))
plt.xlabel('Epoch')
plt.ylabel('Learning Rate in Log10 Scale')
plt.title('Learning Rate over Epochs')

In [ ]:
hist = model.fit_generator(train_generator, 
                           validation_data=val_generator, 
                           epochs=nb_epochs, 
                           callbacks=[chkp, lr_schedule]).history
np.savez('inception_dd_history.npz', loss=hist['loss'], acc=hist['acc'], val_loss=hist['val_loss'], val_acc=hist['val_acc'])

In [ ]:
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
train_loss = plt.plot(hist['loss'], label='train loss')
val_loss = plt.plot(hist['val_loss'], label='val loss')
plt.legend()
plt.title('Loss')

plt.subplot(1,2,2)
train_loss = plt.plot(hist['acc'], label='train acc')
val_loss = plt.plot(hist['val_acc'], label='val acc')
plt.legend()
plt.title('Accuracy')

In [ ]:
# Predictions
# val_generator.filenames

In [ ]:
val_preds = model.predict_generator(generator=val_generator)

In [ ]:
val_preds_class = val_preds.argmax(axis=1)
val_preds_df = pd.DataFrame({'image':val_generator.filenames, 'prediction':val_preds_class})
val_preds_df.head(10)

In [ ]:
val_generator.class_indices

In [ ]:
# incorrect predictions
val_incorrect_pred = [i for i in val_preds_df.index.values if int(val_preds_df.image.values[i].split('/')[0][1])!=val_preds_df.prediction.values[i]]
# val_incorrect_pred = [i for i,j in zip(val_preds_df.image.values,val_preds_df.prediction.values) if int(i.split('/')[0][1])!=j]

In [ ]:
val_incorrect_pred

In [ ]:
plt.figure(figsize=(20,4))
plt.suptitle("Incorrect Predictions", size=32)
for ix,i in enumerate(val_incorrect_pred):
    img = mpimg.imread(os.path.join('val',val_preds_df.image[i]))
    plt.subplot(1,5,ix+1)
    plt.imshow(img)
    plt.axis('off')
    plt.title('Pred: '+class_map['c'+str(val_preds_df.prediction[i])]+'\n'+'Actual: '+class_map[val_preds_df.image[i].split('/')[0]])
    if ix==4:
        break

In [ ]:
# Image Augmentation


In [80]:
# step1 :Image transformation
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

ImportError: cannot import name 'img_as_float32'

In [31]:
!pip install --upgrade scikit-image


  Found existing installation: dask 0.17.5
    Uninstalling dask-0.17.5:
      Successfully uninstalled dask-0.17.5
  Found existing installation: scikit-image 0.13.1
    Uninstalling scikit-image-0.13.1:
      Successfully uninstalled scikit-image-0.13.1


distributed 1.21.8 requires msgpack, which is not installed.
Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\prash\\anaconda3\\lib\\site-packages\\~kimage\\_shared\\geometry.cp36-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [83]:
# step 2 List all the files in a folder and read them
import random
import os

# our folder path containing some images
folder_path = 'E:/Insofe/Python Lab/10_categories'
# the number of file to generate
num_files_desired = 1000

# loop on all files of the folder and build a list of files paths
images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
num_generated_files = 0
while num_generated_files <= num_files_desired:
    # random image from the folder
    image_path = random.choice(images)
    # read image as an two dimensional array of pixels
    image_to_transform = sk.io.imread(image_path)



IndexError: Cannot choose from an empty sequence

In [ ]:
#Step 3 — Images transformation
# dictionary of the transformations functions we defined earlier
available_transformations = {
    'rotate': random_rotation,
    'noise': random_noise,
    'horizontal_flip': horizontal_flip
}

# random num of transformations to apply
num_transformations_to_apply = random.randint(1, len(available_transformations))

num_transformations = 0
transformed_image = None
while num_transformations <= num_transformations_to_apply:
    # choose a random transformation to apply for a single image
    key = random.choice(list(available_transformations))
    transformed_image = available_transformations[key](image_to_transform)

In [ ]:
#Step 4 — Save the new images

# define a name for our new file
new_file_path = '%s/augmented_image_%s.jpg' % (folder_path, num_generated_files)

# write image to the disk
sk.io.imsave(new_file_path, transformed_image)